---
title: "Pan-UKB Pleitropy of Diseases v01"
author: "Saikat Banerjee"
format:
  html: default
date: "2023-12-11"
file-modified: "2023-12-08"
abstract: "We look at the pleiotropy of some common diseases from the Pan-UKB data."
website:
  sidebar:
    collapse-level: 3
    
---

# Getting Setup

In [1]:
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
from pymir import mpl_stylesheet
from pymir import mpl_utils
mpl_stylesheet.banskt_presentation(splinecolor = 'black', dpi = 120, colors = 'kelly')
from matplotlib.gridspec import GridSpec

from nnwmf.optimize import IALM
from nnwmf.optimize import FrankWolfe, FrankWolfe_CV
from nnwmf.utils import model_errors as merr

import sys
sys.path.append("../utils/")
import simulate as mpy_simulate

# Load data and results

In [2]:
data_dir = "/gpfs/commons/home/sbanerjee/work/npd/PanUKB/data"
result_dir = "/gpfs/commons/home/sbanerjee/work/npd/PanUKB/nnwmf"
zscore_filename = f"{data_dir}/GWAS_Zscore.tsv"
trait_filename = f"{data_dir}/trait_manifest_TableS6_no_readme.tsv"
zscore_df = pd.read_csv(zscore_filename, sep = '\t')
trait_df = pd.read_csv(trait_filename, sep = '\t')

# remove extra columns from trait_df

colnames = trait_df.columns.tolist()
colnames[0] = "zindex"
trait_df.columns = colnames
trait_df_mod = trait_df.drop(labels = ['coding', 'modifier', 'coding_description', 'filename', 'aws_link'], axis=1)
trait_df_mod['trait_name'] = trait_df_mod['description']
trait_df_mod['trait_name'] = trait_df_mod['trait_name'].fillna(trait_df_mod['phenocode'])

In [3]:
X_nan = np.array(zscore_df.loc[:, zscore_df.columns!='rsid']).T
X_nan_cent = X_nan - np.nanmean(X_nan, axis = 0, keepdims = True)
X_nan_mask = np.isnan(X_nan)
X_cent = np.nan_to_num(X_nan_cent, copy = True, nan = 0.0)

print (f"We have {X_cent.shape[0]} samples (phenotypes) and {X_cent.shape[1]} features (variants)")
print (f"Fraction of Nan entries: {np.sum(X_nan_mask) / np.prod(X_cent.shape):.3f}")

We have 2483 samples (phenotypes) and 51399 features (variants)
Fraction of Nan entries: 0.000


In [4]:
mf_methods = ['ialm', 'nnm_sparse', 'tsvd']

method_prefix = {
    'ialm' : 'ialm_maxiter10000_admm',
    'nnm_sparse' : 'nnm_sparse_maxiter1000'
}

method_names = {
    'tsvd' : 'Raw Data',
    'ialm' : 'RPCA-IALM',
    'nnm'  : 'NNM-FW',
    'nnm_weighted' : 'NNM-Weighted',
    'nnm_sparse' : 'NNM-Sparse-FW',
}

with open (f"{result_dir}/{method_prefix['ialm']}_progress.pkl", 'rb') as handle:
    ialm = pickle.load(handle)
    
with open (f"{result_dir}/{method_prefix['nnm_sparse']}_progress.pkl", 'rb') as handle:
    nnm_sparse = pickle.load(handle)

In [5]:
def get_principal_components(X):
    X_cent = mpy_simulate.do_standardize(X, scale = False)
    X_cent /= np.sqrt(np.prod(X_cent.shape))
    U, S, Vt = np.linalg.svd(X_cent, full_matrices = False)
    pcomps = U @ np.diag(S)
    loadings = Vt.T @ np.diag(S)
    return loadings, pcomps, S

lowrank_X = dict()
loadings  = dict()
pcomps    = dict()
eigenvals = dict()

for m in mf_methods:
    if m != 'tsvd':
        with open (f"{result_dir}/{method_prefix[m]}_lowrank_X.pkl", 'rb') as handle:
            lowrank_X[m] = pickle.load(handle)
lowrank_X['tsvd'] = X_cent.copy()
for m in mf_methods:
    loadings[m], pcomps[m], eigenvals[m] = get_principal_components(lowrank_X[m])

In [6]:
def get_cos2_scores(pcomps):
    ntrait, npcomp = pcomps.shape
    x = np.zeros((ntrait, npcomp))
    std_pcomps = pcomps / np.sqrt(np.var(pcomps[:,:], axis = 0))
    for i in range(ntrait):
        cos2_trait = np.array([np.square(std_pcomps[i, pcidx]) for pcidx in range(npcomp)])
        x[i, :] = cos2_trait / np.sum(cos2_trait)
    return x

def get_contribution_scores(pcomps):
    ntrait, npcomp = pcomps.shape
    x = np.zeros((ntrait, npcomp))
    std_pcomps = pcomps / np.sqrt(np.var(pcomps[:,:], axis = 0)).reshape(1, -1)
    for i in range(npcomp):
        trait_contr = np.array([np.square(std_pcomps[j, i]) for j in range(ntrait)])
        x[:, i] = trait_contr / np.sum(trait_contr)
    return x

npcomp = 100
cos2_scores = dict()
for m in mf_methods:
    cos2_scores[m] = get_cos2_scores(pcomps[m][:,:npcomp])

trait_contributions = dict()
for m in mf_methods:
    trait_contributions[m] = get_contribution_scores(pcomps[m][:,:npcomp])

# Pleiotropy

In the following, we look at the pleiotropy of different diseases reported in the UKB project.
Given a phenotype (focal trait), we find the hidden factor with the highest contribution to the focal trait. 
We then report the other phenotypes which contribute to that hidden factor. 

*Function* `pan_ukb_pleiotropy(trait_index)`

**Input**: 
  - Index of the focal trait as found in the [manifest file](https://docs.google.com/spreadsheets/d/16OqNQf8cSk2szlmQWYlNOj30NHMjojyf).
  - Number of pleiotropic traits to be reported (optional, default 15).

**Output**: 
  - Index of the top hidden factor.
  - Contribution of the hidden factor on the focal trait.
  - List of pleiotropic traits, and their contributions to the hidden factor.
  
**Global Variables**: `trait_df_mod`, `mf_methods`, `cos2_scores`, `trait_contributions`, `method_names`

In [95]:
from IPython.display import Markdown, display
from IPython.core.display import HTML

import re
def titlecase(s):
    return re.sub(r"[A-Za-z]+('[A-Za-z]+)?", lambda mo: mo.group(0).capitalize(),s)

def print_markdown(string, color = None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))
    
def print_header(string):
    headerstr=f"<h2>{string}</h2>"
    display(Markdown(headerstr))
    
def print_table(list1, list2):
    #table_css = 'table {align:left;display:block}'
    tablestr = "<table style={float:left; align:left;}>"
    for l1, l2 in zip(list1, list2):
        tablestr += f"<tr><td>{l1:.4f}</td><td>{l2}</td></tr>"
    tablestr +="</table>"
    display(Markdown(tablestr))
            
def pan_ukb_pleiotropy(znum, num_trait = 15):
    zindex = znum - 1
    focal_trait_name = trait_df_mod.loc[zindex, 'trait_name']
    print_header(titlecase(focal_trait_name))
    for m in mf_methods:
        print_markdown (f"**{method_names[m]}**", color = "#c8635a")
        top_factor = np.argsort(cos2_scores[m][zindex,:])[::-1][0]
        print_markdown (f"Top Factor: {top_factor + 1} (Contribution: {cos2_scores[m][zindex,top_factor]:.4f})")
        top_traits = np.argsort(trait_contributions[m][:,top_factor])[::-1][:num_trait]
        trait_names = trait_df_mod.loc[top_traits]['trait_name'].tolist()
        print_table(trait_contributions[m][top_traits, top_factor], trait_names)

def pan_ukb_pleiotropy_(znum, num_trait = 15):
    zindex = znum - 1
    focal_trait_name = trait_df_mod.loc[zindex, 'trait_name']
    printheader(titlecase(focal_trait_name))
    for m in mf_methods:
        printmd (f"**{method_names[m]}**", color = "#c8635a")
        top_factor = np.argsort(cos2_scores[m][zindex,:])[::-1][0]
        printmd(f"Top Factor: {top_factor + 1} (Contribution: {cos2_scores[m][zindex,top_factor]:.4f})")
        top_traits = np.argsort(trait_contributions[m][:,top_factor])[::-1][:num_trait]
        trait_names = trait_df_mod.loc[top_traits]['trait_name'].tolist()
        for i, x in zip(top_traits, trait_names):
            print(f"    {trait_contributions[m][i, top_factor]:.4f}   {x}")
        print(f"")

# Common Diseases

In [96]:
pan_ukb_pleiotropy(930)

<h2>Rheumatoid Arthritis</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 47 (Contribution: 0.1697)</span>

<table style={float:left; align:left;}><tr><td>0.0236</td><td>methotrexate</td></tr><tr><td>0.0236</td><td>anti-metabolite</td></tr><tr><td>0.0167</td><td>folic acid supplement|Food Supplement</td></tr><tr><td>0.0167</td><td>folic acid</td></tr><tr><td>0.0146</td><td>Forced expiratory volume in 1-second (FEV1), Best measure</td></tr><tr><td>0.0141</td><td>Forced expiratory volume in 1-second (FEV1)</td></tr><tr><td>0.0131</td><td>Forced vital capacity (FVC), Best measure</td></tr><tr><td>0.0126</td><td>Forced vital capacity (FVC)</td></tr><tr><td>0.0113</td><td>anti-inflammatory</td></tr><tr><td>0.0113</td><td>sulfasalazine</td></tr><tr><td>0.0104</td><td>Rheumatoid arthritis</td></tr><tr><td>0.0097</td><td>Neutrophill percentage</td></tr><tr><td>0.0097</td><td>Rheumatoid arthritis and other inflammatory polyarthropathies</td></tr><tr><td>0.0095</td><td>Corneal resistance factor (left)</td></tr><tr><td>0.0093</td><td>Corneal resistance factor (right)</td></tr></table>

<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 88 (Contribution: 0.0664)</span>

<table style={float:left; align:left;}><tr><td>0.0319</td><td>Whole body fat-free mass</td></tr><tr><td>0.0251</td><td>Leg fat percentage (right)</td></tr><tr><td>0.0194</td><td>Hand grip strength (left)</td></tr><tr><td>0.0120</td><td>Leg predicted mass (right)</td></tr><tr><td>0.0118</td><td>Forced vital capacity (FVC), Best measure</td></tr><tr><td>0.0110</td><td>codine/paracetamol</td></tr><tr><td>0.0103</td><td>Arm fat-free mass (right)</td></tr><tr><td>0.0099</td><td>Mean arterial pressure, automated reading</td></tr><tr><td>0.0096</td><td>Trunk fat-free mass</td></tr><tr><td>0.0096</td><td>Peak expiratory flow (PEF)</td></tr><tr><td>0.0092</td><td>Trunk predicted mass</td></tr><tr><td>0.0090</td><td>Basal metabolic rate</td></tr><tr><td>0.0089</td><td>Diastolic blood pressure, combined automated + manual reading, adjusted by medication</td></tr><tr><td>0.0071</td><td>Arm fat mass (left)</td></tr><tr><td>0.0069</td><td>Age completed full time education</td></tr></table>

<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 73 (Contribution: 0.1319)</span>

<table style={float:left; align:left;}><tr><td>0.0213</td><td>I35 Nonrheumatic aortic valve disorders</td></tr><tr><td>0.0211</td><td>Nonrheumatic aortic valve disorders</td></tr><tr><td>0.0173</td><td>Congenital anomalies of great vessels</td></tr><tr><td>0.0157</td><td>Cardiac congenital anomalies</td></tr><tr><td>0.0147</td><td>Cardiac and circulatory congenital anomalies</td></tr><tr><td>0.0137</td><td>Albumin</td></tr><tr><td>0.0128</td><td>anti-metabolite</td></tr><tr><td>0.0128</td><td>methotrexate</td></tr><tr><td>0.0126</td><td>Heart valve disorders</td></tr><tr><td>0.0094</td><td>Aortic valve disease</td></tr><tr><td>0.0093</td><td>Rheumatoid arthritis</td></tr><tr><td>0.0090</td><td>Rheumatoid arthritis and other inflammatory polyarthropathies</td></tr><tr><td>0.0080</td><td>Osteoporosis</td></tr><tr><td>0.0080</td><td>Osteoporosis NOS</td></tr><tr><td>0.0080</td><td>Treatment/medication code</td></tr></table>

In [97]:
pan_ukb_pleiotropy(420)

<h2>Cancer Of Prostate</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 78 (Contribution: 0.0875)</span>

<table style={float:left; align:left;}><tr><td>0.0272</td><td>Contracture of palmar fascia [Dupuytren's disease]</td></tr><tr><td>0.0262</td><td>Disorders of muscle, ligament, and fascia</td></tr><tr><td>0.0243</td><td>Fasciitis</td></tr><tr><td>0.0186</td><td>Malignant neoplasm of bladder</td></tr><tr><td>0.0182</td><td>C67 Malignant neoplasm of bladder</td></tr><tr><td>0.0157</td><td>Cancer of bladder</td></tr><tr><td>0.0156</td><td>M72 Fibroblastic disorders</td></tr><tr><td>0.0131</td><td>K76 Other diseases of liver</td></tr><tr><td>0.0130</td><td>Other disorders of liver</td></tr><tr><td>0.0127</td><td>Sebaceous cyst</td></tr><tr><td>0.0125</td><td>Diseases of sebaceous glands</td></tr><tr><td>0.0114</td><td>Prostatitis</td></tr><tr><td>0.0112</td><td>Chronic liver disease and cirrhosis</td></tr><tr><td>0.0110</td><td>L72 Follicular cysts of skin and subcutaneous tissue</td></tr><tr><td>0.0107</td><td>Cancer of urinary organs (incl. kidney and bladder)</td></tr></table>

<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 48 (Contribution: 0.0815)</span>

<table style={float:left; align:left;}><tr><td>0.0247</td><td>Weight</td></tr><tr><td>0.0190</td><td>Whole body water mass</td></tr><tr><td>0.0134</td><td>Trunk fat mass</td></tr><tr><td>0.0132</td><td>Age at first live birth</td></tr><tr><td>0.0128</td><td>Weight</td></tr><tr><td>0.0123</td><td>Arm fat-free mass (left)</td></tr><tr><td>0.0119</td><td>Overweight, obesity and other hyperalimentation</td></tr><tr><td>0.0112</td><td>Body mass index (BMI)</td></tr><tr><td>0.0108</td><td>Leg fat percentage (right)</td></tr><tr><td>0.0089</td><td>Hip circumference</td></tr><tr><td>0.0088</td><td>Leg fat-free mass (right)</td></tr><tr><td>0.0081</td><td>Trunk predicted mass</td></tr><tr><td>0.0064</td><td>aspirin</td></tr><tr><td>0.0064</td><td>Leg fat-free mass (left)</td></tr><tr><td>0.0063</td><td>sodium alginate / potassium bicarbonate</td></tr></table>

<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 54 (Contribution: 0.0663)</span>

<table style={float:left; align:left;}><tr><td>0.0277</td><td>Albumin</td></tr><tr><td>0.0221</td><td>IGF-1</td></tr><tr><td>0.0168</td><td>Calcium</td></tr><tr><td>0.0163</td><td>Alkaline phosphatase</td></tr><tr><td>0.0114</td><td>Corneal resistance factor (left)</td></tr><tr><td>0.0106</td><td>Corneal resistance factor (right)</td></tr><tr><td>0.0098</td><td>Total protein</td></tr><tr><td>0.0089</td><td>Corneal hysteresis (left)</td></tr><tr><td>0.0079</td><td>Corneal hysteresis (right)</td></tr><tr><td>0.0065</td><td>Phosphate</td></tr><tr><td>0.0065</td><td>Home location - east co-ordinate (rounded)</td></tr><tr><td>0.0063</td><td>Urate</td></tr><tr><td>0.0062</td><td>Home location at assessment - east co-ordinate (rounded)</td></tr><tr><td>0.0060</td><td>Other disorders of bladder</td></tr><tr><td>0.0056</td><td>Albumin/Globulin ratio</td></tr></table>

In [98]:
pan_ukb_pleiotropy(410)

<h2>Breast Cancer</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 25 (Contribution: 0.1298)</span>

<table style={float:left; align:left;}><tr><td>0.0221</td><td>Breast cancer [female]</td></tr><tr><td>0.0215</td><td>Breast cancer</td></tr><tr><td>0.0210</td><td>Malignant neoplasm of female breast</td></tr><tr><td>0.0200</td><td>C50 Malignant neoplasm of breast</td></tr><tr><td>0.0167</td><td>Arthropathy NOS</td></tr><tr><td>0.0167</td><td>Other arthropathies</td></tr><tr><td>0.0129</td><td>M17 Gonarthrosis [arthrosis of knee]</td></tr><tr><td>0.0110</td><td>Osteoarthrosis</td></tr><tr><td>0.0108</td><td>Unspecified monoarthritis</td></tr><tr><td>0.0098</td><td>Diseases of esophagus</td></tr><tr><td>0.0095</td><td>Diaphragmatic hernia</td></tr><tr><td>0.0093</td><td>Osteoarthritis; localized</td></tr><tr><td>0.0091</td><td>K44 Diaphragmatic hernia</td></tr><tr><td>0.0090</td><td>Esophagitis, GERD and related diseases</td></tr><tr><td>0.0088</td><td>Malignant neoplasm, other</td></tr></table>

<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 66 (Contribution: 0.0840)</span>

<table style={float:left; align:left;}><tr><td>0.0300</td><td>Leg fat-free mass (right)</td></tr><tr><td>0.0273</td><td>Whole body fat-free mass</td></tr><tr><td>0.0264</td><td>Sitting height</td></tr><tr><td>0.0208</td><td>Arm fat-free mass (left)</td></tr><tr><td>0.0128</td><td>Impedance of leg (left)</td></tr><tr><td>0.0128</td><td>Body mass index (BMI)</td></tr><tr><td>0.0120</td><td>Arm predicted mass (right)</td></tr><tr><td>0.0109</td><td>Whole body water mass</td></tr><tr><td>0.0098</td><td>Type 2 diabetes</td></tr><tr><td>0.0097</td><td>Waist circumference</td></tr><tr><td>0.0097</td><td>Arm predicted mass (left)</td></tr><tr><td>0.0085</td><td>Arm fat-free mass (right)</td></tr><tr><td>0.0081</td><td>E66 Obesity</td></tr><tr><td>0.0078</td><td>Impedance of leg (right)</td></tr><tr><td>0.0074</td><td>lansoprazole</td></tr></table>

<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 48 (Contribution: 0.0812)</span>

<table style={float:left; align:left;}><tr><td>0.0133</td><td>Urate</td></tr><tr><td>0.0109</td><td>Past tobacco smoking</td></tr><tr><td>0.0108</td><td>Breast cancer</td></tr><tr><td>0.0108</td><td>Breast cancer [female]</td></tr><tr><td>0.0101</td><td>Malignant neoplasm of female breast</td></tr><tr><td>0.0099</td><td>C50 Malignant neoplasm of breast</td></tr><tr><td>0.0099</td><td>Smoking status, ever vs never</td></tr><tr><td>0.0090</td><td>Age first had sexual intercourse</td></tr><tr><td>0.0086</td><td>Alkaline phosphatase</td></tr><tr><td>0.0075</td><td>Diseases of esophagus</td></tr><tr><td>0.0072</td><td>Gamma glutamyltransferase</td></tr><tr><td>0.0071</td><td>Aspartate aminotransferase</td></tr><tr><td>0.0070</td><td>Esophagitis, GERD and related diseases</td></tr><tr><td>0.0067</td><td>Duration to first press of snap-button in each round</td></tr><tr><td>0.0062</td><td>Estimated glomerular filtration rate, serum creatinine + cystain C</td></tr></table>

In [99]:
pan_ukb_pleiotropy(458)

<h2>Diabetes Mellitus</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 19 (Contribution: 0.1403)</span>

<table style={float:left; align:left;}><tr><td>0.0364</td><td>Red blood cell (erythrocyte) count</td></tr><tr><td>0.0189</td><td>Glycated haemoglobin (HbA1c)</td></tr><tr><td>0.0178</td><td>Mean corpuscular volume</td></tr><tr><td>0.0159</td><td>Lymphocyte count</td></tr><tr><td>0.0150</td><td>Mean sphered cell volume</td></tr><tr><td>0.0146</td><td>Haemoglobin concentration</td></tr><tr><td>0.0137</td><td>Platelet count</td></tr><tr><td>0.0136</td><td>Haematocrit percentage</td></tr><tr><td>0.0132</td><td>Mean corpuscular haemoglobin</td></tr><tr><td>0.0128</td><td>Type 2 diabetes</td></tr><tr><td>0.0127</td><td>Non-albumin protein</td></tr><tr><td>0.0124</td><td>E11 Non-insulin-dependent diabetes mellitus</td></tr><tr><td>0.0119</td><td>Diabetes mellitus</td></tr><tr><td>0.0108</td><td>Platelet crit</td></tr><tr><td>0.0106</td><td>Neutrophill percentage</td></tr></table>

<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 57 (Contribution: 0.0559)</span>

<table style={float:left; align:left;}><tr><td>0.0178</td><td>Arm fat-free mass (left)</td></tr><tr><td>0.0162</td><td>Trunk fat mass</td></tr><tr><td>0.0144</td><td>Body mass index (BMI)</td></tr><tr><td>0.0140</td><td>codine/paracetamol</td></tr><tr><td>0.0135</td><td>Type 2 diabetes</td></tr><tr><td>0.0131</td><td>Comparative height size at age 10</td></tr><tr><td>0.0107</td><td>tetracycline antibiotic|antibiotic</td></tr><tr><td>0.0095</td><td>Impedance of arm (right)</td></tr><tr><td>0.0092</td><td>Hip circumference</td></tr><tr><td>0.0092</td><td>Diabetes mellitus</td></tr><tr><td>0.0089</td><td>Forced expiratory volume in 1-second (FEV1), Best measure</td></tr><tr><td>0.0082</td><td>ACE inhibitor|anti-hypertensive</td></tr><tr><td>0.0077</td><td>Trunk fat-free mass</td></tr><tr><td>0.0069</td><td>trimethoprim</td></tr><tr><td>0.0069</td><td>Weight</td></tr></table>

<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 26 (Contribution: 0.1503)</span>

<table style={float:left; align:left;}><tr><td>0.1042</td><td>HDL cholesterol</td></tr><tr><td>0.0898</td><td>Apolipoprotein A</td></tr><tr><td>0.0473</td><td>Glycated haemoglobin (HbA1c)</td></tr><tr><td>0.0273</td><td>Albumin/Globulin ratio</td></tr><tr><td>0.0231</td><td>Non-albumin protein</td></tr><tr><td>0.0208</td><td>Platelet crit</td></tr><tr><td>0.0192</td><td>Neutrophill percentage</td></tr><tr><td>0.0175</td><td>Neutrophill count</td></tr><tr><td>0.0154</td><td>Lymphocyte percentage</td></tr><tr><td>0.0138</td><td>IGF-1</td></tr><tr><td>0.0131</td><td>Gamma glutamyltransferase</td></tr><tr><td>0.0112</td><td>Type 2 diabetes</td></tr><tr><td>0.0111</td><td>Diabetes mellitus</td></tr><tr><td>0.0111</td><td>E11 Non-insulin-dependent diabetes mellitus</td></tr><tr><td>0.0109</td><td>Mean platelet (thrombocyte) volume</td></tr></table>

In [100]:
pan_ukb_pleiotropy(529)

<h2>Parkinson's Disease</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 97 (Contribution: 0.0888)</span>

<table style={float:left; align:left;}><tr><td>0.0441</td><td>Psoriasis</td></tr><tr><td>0.0425</td><td>L40 Psoriasis</td></tr><tr><td>0.0403</td><td>Psoriasis and related disorders</td></tr><tr><td>0.0187</td><td>Cardiac arrest</td></tr><tr><td>0.0184</td><td>Cardiac arrest and ventricular fibrillation</td></tr><tr><td>0.0181</td><td>I46 Cardiac arrest</td></tr><tr><td>0.0117</td><td>K90 Intestinal malabsorption</td></tr><tr><td>0.0114</td><td>Intestinal malabsorption (non-celiac)</td></tr><tr><td>0.0109</td><td>Obesity</td></tr><tr><td>0.0108</td><td>E66 Obesity</td></tr><tr><td>0.0097</td><td>Overweight, obesity and other hyperalimentation</td></tr><tr><td>0.0095</td><td>Aortic aneurysm</td></tr><tr><td>0.0094</td><td>I71 Aortic aneurysm and dissection</td></tr><tr><td>0.0090</td><td>Celiac disease</td></tr><tr><td>0.0085</td><td>Noninflammatory disorders of ovary, fallopian tube, and broad ligament</td></tr></table>

<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 70 (Contribution: 0.0852)</span>

<table style={float:left; align:left;}><tr><td>0.0262</td><td>Trunk fat percentage</td></tr><tr><td>0.0240</td><td>Whole body fat-free mass</td></tr><tr><td>0.0207</td><td>Leg predicted mass (left)</td></tr><tr><td>0.0201</td><td>Arm fat-free mass (left)</td></tr><tr><td>0.0172</td><td>Leg fat percentage (left)</td></tr><tr><td>0.0147</td><td>Leg fat mass (right)</td></tr><tr><td>0.0105</td><td>Leg fat mass (left)</td></tr><tr><td>0.0083</td><td>NSAID|non-steroidal anti-inflammatory drug</td></tr><tr><td>0.0079</td><td>Usual walking pace</td></tr><tr><td>0.0078</td><td>proton pump inhibitor|PPI</td></tr><tr><td>0.0073</td><td>Impedance of leg (left)</td></tr><tr><td>0.0071</td><td>Estimated glomerular filtration rate, serum creatinine + cystain C</td></tr><tr><td>0.0065</td><td>Arm fat mass (right)</td></tr><tr><td>0.0061</td><td>Diastolic blood pressure, combined automated + manual reading, adjusted by medication</td></tr><tr><td>0.0059</td><td>codine/paracetamol</td></tr></table>

<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 92 (Contribution: 0.1208)</span>

<table style={float:left; align:left;}><tr><td>0.0149</td><td>Urinary calculus</td></tr><tr><td>0.0149</td><td>N20 Calculus of kidney and ureter</td></tr><tr><td>0.0147</td><td>Maximum heart rate during fitness test</td></tr><tr><td>0.0146</td><td>ECG, load</td></tr><tr><td>0.0146</td><td>Maximum workload during fitness test</td></tr><tr><td>0.0139</td><td>Pulse rate, automated reading</td></tr><tr><td>0.0138</td><td>ECG, heart rate</td></tr><tr><td>0.0130</td><td>ECG, number of stages in a phase</td></tr><tr><td>0.0126</td><td>Number of trend entries</td></tr><tr><td>0.0117</td><td>ECG, phase time</td></tr><tr><td>0.0110</td><td>Calculus of ureter</td></tr><tr><td>0.0102</td><td>Processed meat intake</td></tr><tr><td>0.0101</td><td>Calculus of kidney</td></tr><tr><td>0.0088</td><td>Beef intake</td></tr><tr><td>0.0080</td><td>Lamb/mutton intake</td></tr></table>

In [101]:
pan_ukb_pleiotropy(504)

<h2>Dementias</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 99 (Contribution: 0.1643)</span>

<table style={float:left; align:left;}><tr><td>0.0148</td><td>Cardiac arrest</td></tr><tr><td>0.0146</td><td>Cardiac arrest and ventricular fibrillation</td></tr><tr><td>0.0144</td><td>I46 Cardiac arrest</td></tr><tr><td>0.0130</td><td>Other disorders of pancreatic internal secretion</td></tr><tr><td>0.0125</td><td>Hypoglycemia</td></tr><tr><td>0.0122</td><td>E16 Other disorders of pancreatic internal secretion</td></tr><tr><td>0.0112</td><td>Obesity</td></tr><tr><td>0.0111</td><td>E66 Obesity</td></tr><tr><td>0.0104</td><td>E21 Hyperparathyroidism and other disorders of parathyroid gland</td></tr><tr><td>0.0104</td><td>G47 Sleep disorders</td></tr><tr><td>0.0103</td><td>Hyperparathyroidism</td></tr><tr><td>0.0101</td><td>Sleep disorders</td></tr><tr><td>0.0101</td><td>Overweight, obesity and other hyperalimentation</td></tr><tr><td>0.0090</td><td>Varicose veins of lower extremity</td></tr><tr><td>0.0088</td><td>Disorders of parathyroid gland</td></tr></table>

<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 74 (Contribution: 0.0601)</span>

<table style={float:left; align:left;}><tr><td>0.0203</td><td>Number of self-reported non-cancer illnesses</td></tr><tr><td>0.0169</td><td>Leg predicted mass (left)</td></tr><tr><td>0.0162</td><td>Waist circumference</td></tr><tr><td>0.0160</td><td>Hip circumference</td></tr><tr><td>0.0139</td><td>Mean arterial pressure, combined automated + manual reading</td></tr><tr><td>0.0136</td><td>Arm fat mass (left)</td></tr><tr><td>0.0130</td><td>Diastolic blood pressure, combined automated + manual reading</td></tr><tr><td>0.0129</td><td>NSAID|non-steroidal anti-inflammatory drug</td></tr><tr><td>0.0129</td><td>Arm fat-free mass (left)</td></tr><tr><td>0.0128</td><td>Systolic blood pressure, combined automated + manual reading, adjusted by medication</td></tr><tr><td>0.0128</td><td>Impedance of arm (left)</td></tr><tr><td>0.0109</td><td>Essential hypertension</td></tr><tr><td>0.0106</td><td>Body mass index (BMI)</td></tr><tr><td>0.0098</td><td>Diastolic blood pressure, combined automated + manual reading, adjusted by medication</td></tr><tr><td>0.0092</td><td>Peak expiratory flow (PEF)</td></tr></table>

<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 61 (Contribution: 0.0862)</span>

<table style={float:left; align:left;}><tr><td>0.0324</td><td>Heel bone mineral density (BMD)</td></tr><tr><td>0.0322</td><td>Heel bone mineral density (BMD) T-score, automated</td></tr><tr><td>0.0322</td><td>Heel quantitative ultrasound index (QUI), direct entry</td></tr><tr><td>0.0258</td><td>Heel Broadband ultrasound attenuation, direct entry</td></tr><tr><td>0.0139</td><td>Urate</td></tr><tr><td>0.0119</td><td>Heel bone mineral density (BMD) T-score, automated (left)</td></tr><tr><td>0.0119</td><td>Heel quantitative ultrasound index (QUI), direct entry (left)</td></tr><tr><td>0.0119</td><td>Heel broadband ultrasound attenuation (left)</td></tr><tr><td>0.0116</td><td>Heel bone mineral density (BMD) (left)</td></tr><tr><td>0.0115</td><td>Heel bone mineral density (BMD) T-score, automated (right)</td></tr><tr><td>0.0115</td><td>Heel quantitative ultrasound index (QUI), direct entry (right)</td></tr><tr><td>0.0115</td><td>Heel broadband ultrasound attenuation (right)</td></tr><tr><td>0.0114</td><td>Heel bone mineral density (BMD) (right)</td></tr><tr><td>0.0103</td><td>Cerebrovascular disease</td></tr><tr><td>0.0102</td><td>Creatinine</td></tr></table>

In [102]:
pan_ukb_pleiotropy(61)

<h2>G47 Sleep Disorders</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 86 (Contribution: 0.1228)</span>

<table style={float:left; align:left;}><tr><td>0.0469</td><td>Acquired foot deformities</td></tr><tr><td>0.0468</td><td>M20 Acquired deformities of fingers and toes</td></tr><tr><td>0.0290</td><td>Hemoptysis</td></tr><tr><td>0.0286</td><td>R04 Haemorrhage from respiratory passages</td></tr><tr><td>0.0275</td><td>Obesity</td></tr><tr><td>0.0272</td><td>E66 Obesity</td></tr><tr><td>0.0272</td><td>Hallux valgus (Bunion)</td></tr><tr><td>0.0253</td><td>Acquired toe deformities</td></tr><tr><td>0.0248</td><td>Abnormal sputum</td></tr><tr><td>0.0247</td><td>Overweight, obesity and other hyperalimentation</td></tr><tr><td>0.0123</td><td>G47 Sleep disorders</td></tr><tr><td>0.0119</td><td>Sleep disorders</td></tr><tr><td>0.0111</td><td>Hammer toe (acquired)</td></tr><tr><td>0.0110</td><td>Epistaxis or throat hemorrhage</td></tr><tr><td>0.0095</td><td>Prostatitis</td></tr></table>

<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 43 (Contribution: 0.0683)</span>

<table style={float:left; align:left;}><tr><td>0.0234</td><td>ACE inhibitor|anti-hypertensive</td></tr><tr><td>0.0141</td><td>High light scatter reticulocyte count</td></tr><tr><td>0.0133</td><td>Usual walking pace</td></tr><tr><td>0.0121</td><td>Weight</td></tr><tr><td>0.0119</td><td>Leg predicted mass (left)</td></tr><tr><td>0.0109</td><td>Trunk fat-free mass</td></tr><tr><td>0.0104</td><td>Estimated glomerular filtration rate, serum creatinine + cystain C</td></tr><tr><td>0.0100</td><td>Reticulocyte count</td></tr><tr><td>0.0092</td><td>Trunk fat percentage</td></tr><tr><td>0.0091</td><td>Waist circumference</td></tr><tr><td>0.0081</td><td>Age first had sexual intercourse</td></tr><tr><td>0.0077</td><td>Seated height</td></tr><tr><td>0.0066</td><td>Treatment/medication code</td></tr><tr><td>0.0064</td><td>Other arthropathies</td></tr><tr><td>0.0061</td><td>Ankle spacing width (right)</td></tr></table>

<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 91 (Contribution: 0.1037)</span>

<table style={float:left; align:left;}><tr><td>0.0136</td><td>Aspartate aminotransferase</td></tr><tr><td>0.0125</td><td>Platelet distribution width</td></tr><tr><td>0.0112</td><td>Alanine aminotransferase</td></tr><tr><td>0.0084</td><td>Inflammatory bowel disease and other gastroenteritis and colitis</td></tr><tr><td>0.0074</td><td>Spherical power (right)</td></tr><tr><td>0.0070</td><td>Genital prolapse</td></tr><tr><td>0.0070</td><td>Prolapse of vaginal walls</td></tr><tr><td>0.0069</td><td>Spherical power (left)</td></tr><tr><td>0.0068</td><td>N81 Female genital prolapse</td></tr><tr><td>0.0068</td><td>Open wounds of head; neck; and trunk</td></tr><tr><td>0.0063</td><td>C79 Secondary malignant neoplasm of other sites</td></tr><tr><td>0.0061</td><td>E87 Other disorders of fluid, electrolyte and acid-base balance</td></tr><tr><td>0.0061</td><td>Ulcerative colitis</td></tr><tr><td>0.0061</td><td>Electrolyte imbalance</td></tr><tr><td>0.0061</td><td>I77 Other disorders of arteries and arterioles</td></tr></table>

In [103]:
pan_ukb_pleiotropy(59)

<h2>G43 Migraine</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 80 (Contribution: 0.0792)</span>

<table style={float:left; align:left;}><tr><td>0.0312</td><td>Appendicitis</td></tr><tr><td>0.0307</td><td>Acute appendicitis</td></tr><tr><td>0.0287</td><td>Appendiceal conditions</td></tr><tr><td>0.0284</td><td>K35 Acute appendicitis</td></tr><tr><td>0.0208</td><td>Sebaceous cyst</td></tr><tr><td>0.0203</td><td>Diseases of sebaceous glands</td></tr><tr><td>0.0186</td><td>L72 Follicular cysts of skin and subcutaneous tissue</td></tr><tr><td>0.0140</td><td>Contracture of palmar fascia [Dupuytren's disease]</td></tr><tr><td>0.0134</td><td>Disorders of muscle, ligament, and fascia</td></tr><tr><td>0.0133</td><td>Iron deficiency anemias, unspecified or not due to blood loss</td></tr><tr><td>0.0133</td><td>Iron deficiency anemias</td></tr><tr><td>0.0121</td><td>Fasciitis</td></tr><tr><td>0.0118</td><td>D50 Iron deficiency anaemia</td></tr><tr><td>0.0115</td><td>Home location - north co-ordinate (rounded)</td></tr><tr><td>0.0113</td><td>Home location at assessment - north co-ordinate (rounded)</td></tr></table>

<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 46 (Contribution: 0.0576)</span>

<table style={float:left; align:left;}><tr><td>0.0320</td><td>Leg predicted mass (right)</td></tr><tr><td>0.0257</td><td>Whole body fat-free mass</td></tr><tr><td>0.0243</td><td>Leg fat-free mass (right)</td></tr><tr><td>0.0192</td><td>Body mass index (BMI)</td></tr><tr><td>0.0167</td><td>Leg fat mass (left)</td></tr><tr><td>0.0144</td><td>NSAID|non-steroidal anti-inflammatory drug</td></tr><tr><td>0.0120</td><td>simvastatin</td></tr><tr><td>0.0112</td><td>thiazide diuretic|diuretic</td></tr><tr><td>0.0109</td><td>aspirin</td></tr><tr><td>0.0092</td><td>lansoprazole</td></tr><tr><td>0.0091</td><td>ACE inhibitor|anti-hypertensive</td></tr><tr><td>0.0089</td><td>Diastolic blood pressure, automated reading</td></tr><tr><td>0.0085</td><td>Non-cancer illness code, self-reported</td></tr><tr><td>0.0070</td><td>bendroflumethiazide</td></tr><tr><td>0.0070</td><td>amoxicillin</td></tr></table>

<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 56 (Contribution: 0.0733)</span>

<table style={float:left; align:left;}><tr><td>0.0227</td><td>Alkaline phosphatase</td></tr><tr><td>0.0134</td><td>Red blood cell (erythrocyte) distribution width</td></tr><tr><td>0.0104</td><td>Albumin</td></tr><tr><td>0.0079</td><td>Alanine aminotransferase</td></tr><tr><td>0.0077</td><td>Estimated glomerular filtration rate, serum creatinine + cystain C</td></tr><tr><td>0.0072</td><td>Other disorders of bladder</td></tr><tr><td>0.0071</td><td>Aspartate aminotransferase</td></tr><tr><td>0.0071</td><td>Other retinal disorders</td></tr><tr><td>0.0065</td><td>Cystatin C</td></tr><tr><td>0.0064</td><td>N32 Other disorders of bladder</td></tr><tr><td>0.0062</td><td>Calcium</td></tr><tr><td>0.0061</td><td>Gamma glutamyltransferase</td></tr><tr><td>0.0057</td><td>Diabetic retinopathy</td></tr><tr><td>0.0057</td><td>Hereditary retinal dystrophies</td></tr><tr><td>0.0057</td><td>H36 Retinal disorders in diseases classified elsewhere</td></tr></table>

In [104]:
pan_ukb_pleiotropy(591)

<h2>Hypertension</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 2 (Contribution: 0.0982)</span>

<table style={float:left; align:left;}><tr><td>0.0595</td><td>Standing height</td></tr><tr><td>0.0290</td><td>Sitting height</td></tr><tr><td>0.0227</td><td>Seated height</td></tr><tr><td>0.0197</td><td>Forced vital capacity (FVC)</td></tr><tr><td>0.0193</td><td>Forced vital capacity (FVC), Best measure</td></tr><tr><td>0.0169</td><td>Mean arterial pressure, combined automated + manual reading, adjusted by medication</td></tr><tr><td>0.0163</td><td>Systolic blood pressure, combined automated + manual reading, adjusted by medication</td></tr><tr><td>0.0157</td><td>Comparative height size at age 10</td></tr><tr><td>0.0157</td><td>Forced expiratory volume in 1-second (FEV1)</td></tr><tr><td>0.0156</td><td>Mean arterial pressure, automated reading, adjusted by medication</td></tr><tr><td>0.0152</td><td>Forced expiratory volume in 1-second (FEV1), predicted</td></tr><tr><td>0.0150</td><td>Forced expiratory volume in 1-second (FEV1), Best measure</td></tr><tr><td>0.0149</td><td>Systolic blood pressure, automated reading, adjusted by medication</td></tr><tr><td>0.0138</td><td>Trunk fat-free mass</td></tr><tr><td>0.0138</td><td>Trunk predicted mass</td></tr></table>

<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 15 (Contribution: 0.1904)</span>

<table style={float:left; align:left;}><tr><td>0.0670</td><td>Essential hypertension</td></tr><tr><td>0.0596</td><td>Hypertension</td></tr><tr><td>0.0278</td><td>E78 Disorders of lipoprotein metabolism and other lipidaemias</td></tr><tr><td>0.0271</td><td>Disorders of lipoid metabolism</td></tr><tr><td>0.0265</td><td>Hypercholesterolemia</td></tr><tr><td>0.0256</td><td>Hyperlipidemia</td></tr><tr><td>0.0252</td><td>Ischemic Heart Disease</td></tr><tr><td>0.0197</td><td>Other chronic ischemic heart disease, unspecified</td></tr><tr><td>0.0174</td><td>I25 Chronic ischaemic heart disease</td></tr><tr><td>0.0138</td><td>antipyretic</td></tr><tr><td>0.0138</td><td>paracetamol</td></tr><tr><td>0.0134</td><td>opioid analgesic|antipyretic</td></tr><tr><td>0.0130</td><td>Angina pectoris</td></tr><tr><td>0.0111</td><td>E11 Non-insulin-dependent diabetes mellitus</td></tr><tr><td>0.0100</td><td>Leg fat mass (left)</td></tr></table>

<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 3 (Contribution: 0.1210)</span>

<table style={float:left; align:left;}><tr><td>0.0461</td><td>Systolic blood pressure, combined automated + manual reading, adjusted by medication</td></tr><tr><td>0.0425</td><td>Systolic blood pressure, automated reading, adjusted by medication</td></tr><tr><td>0.0405</td><td>Mean arterial pressure, combined automated + manual reading, adjusted by medication</td></tr><tr><td>0.0373</td><td>Mean arterial pressure, automated reading, adjusted by medication</td></tr><tr><td>0.0353</td><td>Systolic blood pressure, combined automated + manual reading</td></tr><tr><td>0.0334</td><td>Systolic blood pressure, automated reading</td></tr><tr><td>0.0304</td><td>Pulse pressure, combined automated + manual reading, adjusted by medication</td></tr><tr><td>0.0288</td><td>Mean arterial pressure, combined automated + manual reading</td></tr><tr><td>0.0282</td><td>Pulse pressure, automated reading, adjusted by medication</td></tr><tr><td>0.0277</td><td>Diastolic blood pressure, combined automated + manual reading, adjusted by medication</td></tr><tr><td>0.0274</td><td>Mean arterial pressure, automated reading</td></tr><tr><td>0.0261</td><td>Diastolic blood pressure, automated reading, adjusted by medication</td></tr><tr><td>0.0246</td><td>Pulse pressure, combined automated + manual reading</td></tr><tr><td>0.0235</td><td>Pulse pressure, automated reading</td></tr><tr><td>0.0180</td><td>Diastolic blood pressure, combined automated + manual reading</td></tr></table>

In [105]:
pan_ukb_pleiotropy(2350)

<h2>Standing Height</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 7 (Contribution: 0.2952)</span>

<table style={float:left; align:left;}><tr><td>0.0748</td><td>Impedance of whole body</td></tr><tr><td>0.0676</td><td>Standing height</td></tr><tr><td>0.0580</td><td>Impedance of arm (left)</td></tr><tr><td>0.0572</td><td>Impedance of arm (right)</td></tr><tr><td>0.0569</td><td>Impedance of leg (left)</td></tr><tr><td>0.0559</td><td>Impedance of leg (right)</td></tr><tr><td>0.0233</td><td>Sitting height</td></tr><tr><td>0.0217</td><td>Seated height</td></tr><tr><td>0.0181</td><td>Comparative height size at age 10</td></tr><tr><td>0.0144</td><td>Forced expiratory volume in 1-second (FEV1), predicted</td></tr><tr><td>0.0132</td><td>Trunk fat percentage</td></tr><tr><td>0.0101</td><td>Body mass index (BMI)</td></tr><tr><td>0.0098</td><td>Body mass index (BMI)</td></tr><tr><td>0.0095</td><td>Reticulocyte count</td></tr><tr><td>0.0094</td><td>Heel quantitative ultrasound index (QUI), direct entry (right)</td></tr></table>

<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 4 (Contribution: 0.1740)</span>

<table style={float:left; align:left;}><tr><td>0.1274</td><td>Standing height</td></tr><tr><td>0.0721</td><td>Sitting height</td></tr><tr><td>0.0263</td><td>Seated height</td></tr><tr><td>0.0192</td><td>Comparative height size at age 10</td></tr><tr><td>0.0190</td><td>Trunk fat-free mass</td></tr><tr><td>0.0189</td><td>Trunk predicted mass</td></tr><tr><td>0.0134</td><td>Whole body fat-free mass</td></tr><tr><td>0.0117</td><td>Whole body water mass</td></tr><tr><td>0.0114</td><td>Waist circumference</td></tr><tr><td>0.0112</td><td>Leg fat mass (left)</td></tr><tr><td>0.0112</td><td>Arm fat mass (right)</td></tr><tr><td>0.0107</td><td>Leg fat mass (right)</td></tr><tr><td>0.0101</td><td>Arm fat mass (left)</td></tr><tr><td>0.0099</td><td>E66 Obesity</td></tr><tr><td>0.0099</td><td>Obesity</td></tr></table>

<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 2 (Contribution: 0.2623)</span>

<table style={float:left; align:left;}><tr><td>0.0640</td><td>Standing height</td></tr><tr><td>0.0351</td><td>Sitting height</td></tr><tr><td>0.0250</td><td>Seated height</td></tr><tr><td>0.0206</td><td>Forced vital capacity (FVC)</td></tr><tr><td>0.0197</td><td>Forced vital capacity (FVC), Best measure</td></tr><tr><td>0.0186</td><td>Comparative height size at age 10</td></tr><tr><td>0.0170</td><td>Forced expiratory volume in 1-second (FEV1), predicted</td></tr><tr><td>0.0161</td><td>Forced expiratory volume in 1-second (FEV1)</td></tr><tr><td>0.0152</td><td>Forced expiratory volume in 1-second (FEV1), Best measure</td></tr><tr><td>0.0132</td><td>Mean arterial pressure, combined automated + manual reading, adjusted by medication</td></tr><tr><td>0.0126</td><td>Systolic blood pressure, combined automated + manual reading, adjusted by medication</td></tr><tr><td>0.0121</td><td>Mean arterial pressure, automated reading, adjusted by medication</td></tr><tr><td>0.0120</td><td>Leg fat percentage (left)</td></tr><tr><td>0.0120</td><td>Leg fat percentage (right)</td></tr><tr><td>0.0118</td><td>Trunk fat-free mass</td></tr></table>

In [106]:
pan_ukb_pleiotropy(1747)

<h2>Intake Of Sugar Added To Coffee</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 52 (Contribution: 0.1127)</span>

<table style={float:left; align:left;}><tr><td>0.0152</td><td>Saturated fat</td></tr><tr><td>0.0138</td><td>Albumin</td></tr><tr><td>0.0124</td><td>Fat</td></tr><tr><td>0.0108</td><td>Osteoporosis</td></tr><tr><td>0.0108</td><td>Osteoporosis NOS</td></tr><tr><td>0.0093</td><td>Vitamin C</td></tr><tr><td>0.0086</td><td>Home location - north co-ordinate (rounded)</td></tr><tr><td>0.0084</td><td>methotrexate</td></tr><tr><td>0.0084</td><td>anti-metabolite</td></tr><tr><td>0.0081</td><td>Retinol</td></tr><tr><td>0.0079</td><td>Home location at assessment - north co-ordinate (rounded)</td></tr><tr><td>0.0072</td><td>Carotene</td></tr><tr><td>0.0071</td><td>Osteoporosis, osteopenia and pathological fracture</td></tr><tr><td>0.0069</td><td>xanthine oxidase inhibitor|anti-gout agent</td></tr><tr><td>0.0069</td><td>allopurinol</td></tr></table>

<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 54 (Contribution: 0.0723)</span>

<table style={float:left; align:left;}><tr><td>0.0427</td><td>Whole body fat-free mass</td></tr><tr><td>0.0224</td><td>NSAID|non-steroidal anti-inflammatory drug</td></tr><tr><td>0.0193</td><td>Leg fat mass (right)</td></tr><tr><td>0.0167</td><td>Usual walking pace</td></tr><tr><td>0.0119</td><td>Z86 Personal history of certain other diseases</td></tr><tr><td>0.0118</td><td>Weight</td></tr><tr><td>0.0100</td><td>clarithromycin</td></tr><tr><td>0.0099</td><td>Arm predicted mass (left)</td></tr><tr><td>0.0085</td><td>Whole body fat mass</td></tr><tr><td>0.0071</td><td>Pulse pressure, combined automated + manual reading, adjusted by medication</td></tr><tr><td>0.0066</td><td>Leg fat mass (left)</td></tr><tr><td>0.0065</td><td>lansoprazole</td></tr><tr><td>0.0063</td><td>aspirin</td></tr><tr><td>0.0063</td><td>diclofenac</td></tr><tr><td>0.0061</td><td>Other mental disorder</td></tr></table>

<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 75 (Contribution: 0.0839)</span>

<table style={float:left; align:left;}><tr><td>0.0341</td><td>SHBG</td></tr><tr><td>0.0298</td><td>Pulse rate, automated reading</td></tr><tr><td>0.0129</td><td>Pulse rate</td></tr><tr><td>0.0122</td><td>Testosterone</td></tr><tr><td>0.0118</td><td>COVID-19 positive (controls include untested)</td></tr><tr><td>0.0118</td><td>COVID-19 positive (controls include untested), only patients from centers in England</td></tr><tr><td>0.0113</td><td>COVID-19 positive (controls include untested)</td></tr><tr><td>0.0112</td><td>COVID-19 positive (controls include untested), only patients from centers in England</td></tr><tr><td>0.0099</td><td>COVID-19 positive (controls only COVID-19 negative)</td></tr><tr><td>0.0097</td><td>Estimated glomerular filtration rate, serum creatinine + cystain C</td></tr><tr><td>0.0096</td><td>COVID-19 positive (controls only COVID-19 negative)</td></tr><tr><td>0.0090</td><td>Cystatin C</td></tr><tr><td>0.0090</td><td>Fasciitis</td></tr><tr><td>0.0089</td><td>Contracture of palmar fascia [Dupuytren's disease]</td></tr><tr><td>0.0088</td><td>Disorders of muscle, ligament, and fascia</td></tr></table>

In [107]:
pan_ukb_pleiotropy(1677)

<h2>Covid-19 Positive (Controls Only Covid-19 Negative)</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 42 (Contribution: 0.4360)</span>

<table style={float:left; align:left;}><tr><td>0.0801</td><td>COVID-19 positive (controls include untested)</td></tr><tr><td>0.0800</td><td>COVID-19 positive (controls include untested), only patients from centers in England</td></tr><tr><td>0.0792</td><td>COVID-19 positive (controls include untested)</td></tr><tr><td>0.0791</td><td>COVID-19 positive (controls include untested), only patients from centers in England</td></tr><tr><td>0.0557</td><td>COVID-19 positive (controls only COVID-19 negative)</td></tr><tr><td>0.0498</td><td>COVID-19 positive (controls only COVID-19 negative)</td></tr><tr><td>0.0135</td><td>Cholelithiasis</td></tr><tr><td>0.0132</td><td>Cholelithiasis and cholecystitis</td></tr><tr><td>0.0130</td><td>K80 Cholelithiasis</td></tr><tr><td>0.0094</td><td>Chronic dermatitis due to solar radiation</td></tr><tr><td>0.0093</td><td>L57 Skin changes due to chronic exposure to nonionising radiation</td></tr><tr><td>0.0082</td><td>Dermatitis due to solar radiation</td></tr><tr><td>0.0069</td><td>Actinic keratosis</td></tr><tr><td>0.0068</td><td>Breast cancer [female]</td></tr><tr><td>0.0066</td><td>Breast cancer</td></tr></table>

<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

<span style='color:None'>Top Factor: 47 (Contribution: 0.0847)</span>

<table style={float:left; align:left;}><tr><td>0.0245</td><td>Age first had sexual intercourse</td></tr><tr><td>0.0170</td><td>Forced expiratory volume in 1-second (FEV1), predicted</td></tr><tr><td>0.0135</td><td>ACE inhibitor|anti-hypertensive</td></tr><tr><td>0.0126</td><td>beta-lactam antibiotic|antibiotic</td></tr><tr><td>0.0115</td><td>Forced expiratory volume in 1-second (FEV1), Best measure</td></tr><tr><td>0.0115</td><td>Whole body fat-free mass</td></tr><tr><td>0.0112</td><td>Arm fat-free mass (left)</td></tr><tr><td>0.0096</td><td>Trunk fat percentage</td></tr><tr><td>0.0093</td><td>Waist circumference</td></tr><tr><td>0.0087</td><td>Leg predicted mass (left)</td></tr><tr><td>0.0085</td><td>Impedance of leg (right)</td></tr><tr><td>0.0081</td><td>Leg fat-free mass (left)</td></tr><tr><td>0.0070</td><td>Ankle spacing width</td></tr><tr><td>0.0069</td><td>Body fat percentage</td></tr><tr><td>0.0059</td><td>Cholesterol</td></tr></table>

<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 67 (Contribution: 0.5227)</span>

<table style={float:left; align:left;}><tr><td>0.0846</td><td>COVID-19 positive (controls include untested), only patients from centers in England</td></tr><tr><td>0.0846</td><td>COVID-19 positive (controls include untested)</td></tr><tr><td>0.0844</td><td>COVID-19 positive (controls include untested), only patients from centers in England</td></tr><tr><td>0.0843</td><td>COVID-19 positive (controls include untested)</td></tr><tr><td>0.0765</td><td>COVID-19 positive (controls only COVID-19 negative)</td></tr><tr><td>0.0720</td><td>COVID-19 positive (controls only COVID-19 negative)</td></tr><tr><td>0.0317</td><td>IGF-1</td></tr><tr><td>0.0114</td><td>Platelet distribution width</td></tr><tr><td>0.0088</td><td>Estimated glomerular filtration rate, serum creatinine + cystain C</td></tr><tr><td>0.0069</td><td>Cystatin C</td></tr><tr><td>0.0047</td><td>Systolic blood pressure, manual reading</td></tr><tr><td>0.0044</td><td>Systolic blood pressure, manual reading, adjusted by medication</td></tr><tr><td>0.0040</td><td>Mean arterial pressure, manual reading</td></tr><tr><td>0.0038</td><td>Estimated glomerular filtration rate, serum creatinine</td></tr><tr><td>0.0037</td><td>Creatinine</td></tr></table>

# Neuropsychiatric Disorders

In [108]:
for z in range(52,58):
    pan_ukb_pleiotropy(z)

<h2>F05 Delirium, Not Induced By Alcohol And Other Psychoactive Substances</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 99 (Contribution: 0.1305)</span>

<table style={float:left; align:left;}><tr><td>0.0148</td><td>Cardiac arrest</td></tr><tr><td>0.0146</td><td>Cardiac arrest and ventricular fibrillation</td></tr><tr><td>0.0144</td><td>I46 Cardiac arrest</td></tr><tr><td>0.0130</td><td>Other disorders of pancreatic internal secretion</td></tr><tr><td>0.0125</td><td>Hypoglycemia</td></tr><tr><td>0.0122</td><td>E16 Other disorders of pancreatic internal secretion</td></tr><tr><td>0.0112</td><td>Obesity</td></tr><tr><td>0.0111</td><td>E66 Obesity</td></tr><tr><td>0.0104</td><td>E21 Hyperparathyroidism and other disorders of parathyroid gland</td></tr><tr><td>0.0104</td><td>G47 Sleep disorders</td></tr><tr><td>0.0103</td><td>Hyperparathyroidism</td></tr><tr><td>0.0101</td><td>Sleep disorders</td></tr><tr><td>0.0101</td><td>Overweight, obesity and other hyperalimentation</td></tr><tr><td>0.0090</td><td>Varicose veins of lower extremity</td></tr><tr><td>0.0088</td><td>Disorders of parathyroid gland</td></tr></table>

<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

...

<span style='color:None'>Top Factor: 93 (Contribution: 0.0847)</span>

<table style={float:left; align:left;}><tr><td>0.0244</td><td>Impedance of arm (right)</td></tr><tr><td>0.0191</td><td>Trunk fat mass</td></tr><tr><td>0.0179</td><td>E11 Non-insulin-dependent diabetes mellitus</td></tr><tr><td>0.0148</td><td>Impedance of leg (right)</td></tr><tr><td>0.0123</td><td>Type 2 diabetes</td></tr><tr><td>0.0104</td><td>Leg fat-free mass (left)</td></tr><tr><td>0.0102</td><td>Leg fat mass (left)</td></tr><tr><td>0.0100</td><td>Leg fat mass (right)</td></tr><tr><td>0.0100</td><td>doxycycline</td></tr><tr><td>0.0100</td><td>Whole body water mass</td></tr><tr><td>0.0090</td><td>Body mass index (BMI)</td></tr><tr><td>0.0087</td><td>Trunk fat percentage</td></tr><tr><td>0.0086</td><td>tetracycline antibiotic|antibiotic</td></tr><tr><td>0.0081</td><td>ramipril</td></tr><tr><td>0.0077</td><td>Cystatin C</td></tr></table>

<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 53 (Contribution: 0.0682)</span>

<table style={float:left; align:left;}><tr><td>0.0102</td><td>Corneal resistance factor (right)</td></tr><tr><td>0.0096</td><td>X61 Intentional self-poisoning by and exposure to antiepileptic, sedative-hypnotic, anti-Parkinsonism and psychotropic drugs, not elsewhere classified</td></tr><tr><td>0.0091</td><td>Corneal resistance factor (left)</td></tr><tr><td>0.0090</td><td>Poisoning by psychotropic agents</td></tr><tr><td>0.0085</td><td>SHBG</td></tr><tr><td>0.0083</td><td>T43 Poisoning by psychotropic drugs, not elsewhere classified</td></tr><tr><td>0.0083</td><td>Suicide or self-inflicted injury</td></tr><tr><td>0.0080</td><td>Suicidal ideation or attempt</td></tr><tr><td>0.0075</td><td>Home location - north co-ordinate (rounded)</td></tr><tr><td>0.0075</td><td>Home location at assessment - north co-ordinate (rounded)</td></tr><tr><td>0.0074</td><td>Toxic effect of (non-ethyl) alcohol and petroleum and other solvents</td></tr><tr><td>0.0073</td><td>T51 Toxic effect of alcohol</td></tr><tr><td>0.0073</td><td>T39 Poisoning by nonopioid analgesics, antipyretics and antirheumatics</td></tr><tr><td>0.0073</td><td>Corneal hysteresis (right)</td></tr><tr><td>0.0072</td><td>X60 Intentional self-poisoning by and exposure to nonopioid analgesics, antipyretics and antirheumatics</td></tr></table>

In [109]:
for z in range(520,538):
    pan_ukb_pleiotropy(z)

<h2>Other Mental Disorder</h2>

<span style='color:#c8635a'>**RPCA-IALM**</span>

<span style='color:None'>Top Factor: 6 (Contribution: 0.1623)</span>

<table style={float:left; align:left;}><tr><td>0.0290</td><td>Trunk fat percentage</td></tr><tr><td>0.0231</td><td>Body fat percentage</td></tr><tr><td>0.0229</td><td>Impedance of whole body</td></tr><tr><td>0.0226</td><td>Impedance of arm (right)</td></tr><tr><td>0.0224</td><td>Impedance of arm (left)</td></tr><tr><td>0.0194</td><td>Trunk fat mass</td></tr><tr><td>0.0155</td><td>Arm fat percentage (left)</td></tr><tr><td>0.0154</td><td>Whole body fat mass</td></tr><tr><td>0.0153</td><td>Arm fat percentage (right)</td></tr><tr><td>0.0139</td><td>Leg fat percentage (right)</td></tr><tr><td>0.0133</td><td>Leg fat percentage (left)</td></tr><tr><td>0.0132</td><td>Impedance of leg (left)</td></tr><tr><td>0.0129</td><td>Impedance of leg (right)</td></tr><tr><td>0.0090</td><td>Leg fat mass (right)</td></tr><tr><td>0.0088</td><td>Arm fat mass (left)</td></tr></table>

<span style='color:#c8635a'>**NNM-Sparse-FW**</span>

...

<span style='color:None'>Top Factor: 96 (Contribution: 0.1300)</span>

<table style={float:left; align:left;}><tr><td>0.0249</td><td>Ankle spacing width (right)</td></tr><tr><td>0.0187</td><td>Impedance of leg (left)</td></tr><tr><td>0.0154</td><td>proton pump inhibitor|PPI</td></tr><tr><td>0.0153</td><td>Angina pectoris</td></tr><tr><td>0.0143</td><td>Whole body fat-free mass</td></tr><tr><td>0.0134</td><td>Non-cancer illness code, self-reported</td></tr><tr><td>0.0127</td><td>benzodiazepine|sedative|anxiety</td></tr><tr><td>0.0115</td><td>Ischemic Heart Disease</td></tr><tr><td>0.0111</td><td>Sitting height</td></tr><tr><td>0.0088</td><td>Leg predicted mass (left)</td></tr><tr><td>0.0088</td><td>Seated height</td></tr><tr><td>0.0085</td><td>Leg fat mass (left)</td></tr><tr><td>0.0084</td><td>Peak expiratory flow (PEF)</td></tr><tr><td>0.0072</td><td>Arm predicted mass (right)</td></tr><tr><td>0.0070</td><td>Age at first live birth</td></tr></table>

<span style='color:#c8635a'>**Raw Data**</span>

<span style='color:None'>Top Factor: 61 (Contribution: 0.0849)</span>

<table style={float:left; align:left;}><tr><td>0.0324</td><td>Heel bone mineral density (BMD)</td></tr><tr><td>0.0322</td><td>Heel bone mineral density (BMD) T-score, automated</td></tr><tr><td>0.0322</td><td>Heel quantitative ultrasound index (QUI), direct entry</td></tr><tr><td>0.0258</td><td>Heel Broadband ultrasound attenuation, direct entry</td></tr><tr><td>0.0139</td><td>Urate</td></tr><tr><td>0.0119</td><td>Heel bone mineral density (BMD) T-score, automated (left)</td></tr><tr><td>0.0119</td><td>Heel quantitative ultrasound index (QUI), direct entry (left)</td></tr><tr><td>0.0119</td><td>Heel broadband ultrasound attenuation (left)</td></tr><tr><td>0.0116</td><td>Heel bone mineral density (BMD) (left)</td></tr><tr><td>0.0115</td><td>Heel bone mineral density (BMD) T-score, automated (right)</td></tr><tr><td>0.0115</td><td>Heel quantitative ultrasound index (QUI), direct entry (right)</td></tr><tr><td>0.0115</td><td>Heel broadband ultrasound attenuation (right)</td></tr><tr><td>0.0114</td><td>Heel bone mineral density (BMD) (right)</td></tr><tr><td>0.0103</td><td>Cerebrovascular disease</td></tr><tr><td>0.0102</td><td>Creatinine</td></tr></table>